In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = ChebConv(in_channels, 32, K=2)
        self.conv2 = ChebConv(32, out_channels, K=2)
        
        self.conv1 = SAGEConv(in_channels, 32)
        self.conv2 = SAGEConv(32, out_channels)
        
#         self.conv1 = GCNConv(in_channels, 32)
#         self.conv2 = GCNConv(32, out_channels)
        
    def forward(self, x, edge_index):
#         x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
model = Net(dataset.num_node_features, dataset.num_classes)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model.to(device)

data = dataset[0]

x = data.x.to(device)
y = data.y.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:

train_loader = NeighborSampler(data.edge_index, node_idx=data.train_mask,
                               sizes=[25, 10], batch_size=1024, shuffle=True,
                               num_workers=12)

In [ ]:
@torch.no_grad()
def test():
    model.eval()

    out = model(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]

    return results

In [ ]:
# evaluate()

In [ ]:
model.train()
for i in range(5):
    
    for batch_size, n_id, adjs in train_loader:
        
        optimizer.zero_grad()

        out = model(x[n_id], adjs)
        
        loss = F.nll_loss(out, y[n_id[:batch_size]])
    
        print(loss)
        
        loss.backward()
        optimizer.step()
        
#     print(loss)
#     print(out[0])

In [ ]:
evaluate()